<a href="https://colab.research.google.com/github/yashna02/Data-Extraction-from-PyMRIO-System/blob/main/Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setting Up**

In [ ]:
pip install git+https://github.com/IndEcol/pymrio@master

  Cloning https://github.com/IndEcol/pymrio (to revision master) to /tmp/pip-req-build-zhywodbz
  Running command git clone --filter=blob:none --quiet https://github.com/IndEcol/pymrio /tmp/pip-req-build-zhywodbz
  Resolved https://github.com/IndEcol/pymrio to commit d24b4346ea38d7de8cfbb75f79a73017316d7e0d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.8 MB/s eta 0:00:00
  Created wheel for pymrio: filename=pymrio-0.5.1-py3-none-any.whl size=138361 sha256=4469496007a427b45eb1d3d25b68b304c3a79a70192e9014dee311c2c4b38ff7
  Stored in directory: /tmp/pip-ephem-wheel-cache-bd8e1_lj/wheels/0b/7b/34/827ad8ecf1b3e5bfc226dfeecd47b9e1dd6c85eb1880669ea8
Successfully built pymrio
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
ERROR: pip's dependency resolve

In [ ]:
import pymrio
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# **Making IO Systems**

For complete understanding, refer https://github.com/yashna02/MRIO-SDA-using-PyMRIO/blob/main/EE-MRIO_SDA.ipynb

In [ ]:
# Define the years range
years = range(2000, 2015)

# Define the sector and region lists
sectors = ["A01","A02","A03","B","C10-C12","C13-C15","C16","C17","C18","C19","C20","C21","C22","C23","C24","C25","C26","C27","C28","C29","C30","C31_C32","C33","D35","E36","E37-E39","F","G45","G46","G47","H49","H50","H51","H52","H53","I","J58","J59_J60","J61","J62_J63","K64","K65","K66","L68","M69_M70","M71","M72","M73","M74_M75","N","O84","P85","Q","R_S","T","U"]
regions = ["AUS","AUT","BEL","BGR","BRA","CAN","CHE","CHN","CYP","CZE","DEU","DNK","ESP","EST","FIN","FRA","GBR","GRC","HRV","HUN","IDN","IND","IRL","ITA","JPN","KOR","LTU","LUX","LVA","MEX","MLT","NOR","POL","PRT","ROU", "ROW", "RUS","SVK","SVN","SWE","TUR","TWN", "USA"]

# Create a dictionary to store the io objects for each year
io_dict = {}

# Iterate over the years
for year in years:
    print(f"Working on {year}")
    # Read the Z file for the current year
    z_file = f"/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/Z_{year}.xlsx"
    z_data = pd.read_excel(z_file, header=0, index_col=0)

    # Create the Z DataFrame with the multi-index
    z_multiindex = pd.MultiIndex.from_product([regions, sectors], names=["region", "sector"])
    z_df = pd.DataFrame(data=z_data.values, index=z_multiindex, columns=z_multiindex)

    # Read the Y file for the current year
    y_file = f"/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/Y_{year}.xlsx"
    y_data = pd.read_excel(y_file, header=0, index_col=0)

    # Create the Y DataFrame with the multi-index
    y_multiindex = pd.MultiIndex.from_product([regions, ["final demand"]], names=["region", "category"])
    y_df = pd.DataFrame(data=y_data.values, index=z_multiindex, columns=y_multiindex)

    # Read the F file for the current year
    f_file = f"/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/F_{year}.xlsx"
    f_data = pd.read_excel(f_file, header=0)

    # Create the F DataFrame
    f_df = pd.DataFrame(data=f_data.values, index=["Emissions"], columns=z_multiindex)

    # Read the V file for the current year
    v_file = f"/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/V_{year}.xlsx"
    v_data = pd.read_excel(v_file, header=0, index_col=0)

    # Create the V DataFrame
    v_df = pd.DataFrame(data=v_data.values, index=["Emissions"], columns=z_multiindex)

    # Create the IOSystem for the current year
    io = pymrio.IOSystem()
    io.Z = z_df
    io.Y = y_df
    io.emissions = pymrio.Extension(name="Emissions", F=f_df)
    io.emissions.unit = pd.DataFrame(data=["Kt"], index=f_data.index, columns=["unit"])
    io.value_added = pymrio.Extension(name="value_added", F=v_df)
    io.value_added.unit = pd.DataFrame(data=["Million USD"], index=v_data.index, columns=["unit"])

    # Calculate all matrices
    io.calc_all()

    # Define the rename dictionary
    rename_dict = {'A01': 'Sec1', 'A02': 'Sec1', 'A03': 'Sec1', 'B': 'Sec2', 'C10-C12': 'Sec3', 'C13-C15': 'Sec3',
                   'C16': 'Sec3', 'C17': 'Sec3', 'C18': 'Sec8', 'C19': 'Sec3', 'C20': 'Sec3', 'C21': 'Sec3',
                   'C22': 'Sec3', 'C23': 'Sec3', 'C24': 'Sec3', 'C25': 'Sec3', 'C26': 'Sec3', 'C27': 'Sec3',
                   'C28': 'Sec3', 'C29': 'Sec3', 'C30': 'Sec3', 'C31_C32': 'Sec3', 'C33': 'Sec8', 'D35': 'Sec4',
                   'E36': 'Sec8', 'E37-E39': 'Sec8', 'F': 'Sec5', 'G45': 'Sec8', 'G46': 'Sec8', 'G47': 'Sec8',
                   'H49': 'Sec6', 'H50': 'Sec6', 'H51': 'Sec6', 'H52': 'Sec8', 'H53': 'Sec8', 'I': 'Sec7',
                   'J58': 'Sec8', 'J59_J60': 'Sec8', 'J61': 'Sec8', 'J62_J63': 'Sec8', 'K64': 'Sec8', 'K65': 'Sec8',
                   'K66': 'Sec8', 'L68': 'Sec8', 'M69_M70': 'Sec8', 'M71': 'Sec8', 'M72': 'Sec8', 'M73': 'Sec8',
                   'M74_M75': 'Sec8', 'N': 'Sec8', 'O84': 'Sec8', 'P85': 'Sec8', 'Q': 'Sec8', 'R_S': 'Sec8',
                   'T': 'Sec7', 'U': 'Sec8'}

    # Rename sectors and aggregate duplicates
    io.rename_sectors(rename_dict)
    io.aggregate_duplicates()

    # Recalculate all matrices
    io.calc_all()

    # Save the IOSystem data to files
    output_path = f"/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Output/io_{year}"
    io.save_all(path=output_path)

    # Store the io object in the dictionary
    io_dict[year] = io


Working on 2000
Working on 2001
Working on 2002
Working on 2003
Working on 2004
Working on 2005
Working on 2006
Working on 2007
Working on 2008
Working on 2009
Working on 2010
Working on 2011
Working on 2012
Working on 2013
Working on 2014


#**Creating dataframe - All sectors**

Contains
*   **PE**: Production based emissions (D_pba_reg from emissions extension)
*   **CE**: Consumption based emissions (D_cba_reg from emissions extension)
*   **EEE**: Emissions embodied in exports (D_exp_reg from emissions extension)
*   **EEI**: Emissions embodied in imports (D_imp_reg from emissions extension)
*   **VAX**: Value added embodied in exports (D_exp_reg from value added extension)
*   **VAM**: Value added embodied in imports (D_imp_reg from value added extension)

Calculate: (as per Kim and Tromp 2021 paper)
* **NE**: Net embodied emissions in exports (EEE-EEI)
* **NV**: Net value added in exports (VAX-VAM)
* **EIE**: Emission intensity of value added exports (EEE/VAX)
* **EIM**: Emission intensity of value added imports (EEI/VAM)
* **NEI**: Net emission intensity (EIE-EIM)




**For the year 2014**


In [ ]:
PE_2014 = io_dict[2014].emissions.D_pba_reg.T
PE_2014.rename(columns={'Emissions': 'PE'}, inplace=True)
CE_2014 = io_dict[2014].emissions.D_cba_reg.T
CE_2014.rename(columns={'Emissions': 'CE'}, inplace=True)
EEE_2014 = io_dict[2014].emissions.D_exp_reg.T
EEE_2014.rename(columns={'Emissions': 'EEE'}, inplace=True)
EEI_2014 = io_dict[2014].emissions.D_imp_reg.T
EEI_2014.rename(columns={'Emissions': 'EEI'}, inplace=True)
VAX_2014 = io_dict[2014].value_added.D_exp_reg.T
VAX_2014.rename(columns={'Emissions': 'VAX'}, inplace=True)
VAM_2014 = io_dict[2014].value_added.D_imp_reg.T
VAM_2014.rename(columns={'Emissions': 'VAM'}, inplace=True)


df_2014 = pd.concat([PE_2014, CE_2014, EEE_2014, EEI_2014, VAX_2014, VAM_2014], axis=1)
df_2014.insert(0, 'Year', 2014)

df_2014['NE'] = df_2014['EEE'] - df_2014['EEI']
df_2014['NV'] = df_2014['VAX'] - df_2014['VAM']
df_2014['EIE'] = df_2014['EEE'] / df_2014['VAX']
df_2014['EIM'] = df_2014['EEI'] / df_2014['VAM']
df_2014['NEI'] = df_2014['EIE'] - df_2014['EIM']

In [ ]:
df_2014

,Year,PE,CE,EEE,EEI,VAX,VAM,NE,NV,EIE,EIM,NEI
region,,,,,,,,,,,,
AUS,2014,365.240602,422.049413,113.685598,170.494409,3.120283e+05,2.194822e+05,-56.808811,9.254608e+04,0.000364,0.000777,-0.000412
AUT,2014,43.508710,65.370065,26.832341,48.693697,1.206412e+05,1.020518e+05,-21.861355,1.858946e+04,0.000222,0.000477,-0.000255
BEL,2014,71.884234,108.230053,48.168620,84.514439,1.720537e+05,1.465313e+05,-36.345819,2.552238e+04,0.000280,0.000577,-0.000297
BGR,2014,43.129553,35.380345,19.640844,11.891636,1.590677e+04,1.961551e+04,7.749208,-3.708740e+03,0.001235,0.000606,0.000629
BRA,2014,494.476953,590.904622,98.040653,194.468322,2.141971e+05,2.586230e+05,-96.427669,-4.442585e+04,0.000458,0.000752,-0.000294
CAN,2014,466.506366,475.158537,199.178037,207.830209,3.937765e+05,3.753868e+05,-8.652171,1.838972e+04,0.000506,0.000554,-0.000048
CHE,2014,23.633138,86.079002,10.573315,73.019179,2.181100e+05,1.609175e+05,-62.445864,5.719252e+04,0.000048,0.000454,-0.000405
CHN,2014,9946.168067,8458.038324,2298.857525,810.727782,1.665082e+06,1.232865e+06,1488.129743,4.322171e+05,0.001381,0.000658,0.000723
CYP,2014,5.433132,8.516590,1.912530,4.995987,5.900527e+03,7.237471e+03,-3.083458,-1.336944e+03,0.000324,0.000690,-0.000366


**For years 2000-2014**

In [ ]:
# Initialize an empty list to store DataFrames for each year
dfs = []

# Iterate through years from 2000 to 2014
for year in range(2000, 2015):
    PE_year = io_dict[year].emissions.D_pba_reg.T
    PE_year.rename(columns={'Emissions': 'PE'}, inplace=True)
    CE_year = io_dict[year].emissions.D_cba_reg.T
    CE_year.rename(columns={'Emissions': 'CE'}, inplace=True)
    EEE_year = io_dict[year].emissions.D_exp_reg.T
    EEE_year.rename(columns={'Emissions': 'EEE'}, inplace=True)
    EEI_year = io_dict[year].emissions.D_imp_reg.T
    EEI_year.rename(columns={'Emissions': 'EEI'}, inplace=True)
    VAX_year = io_dict[year].value_added.D_exp_reg.T
    VAX_year.rename(columns={'Emissions': 'VAX'}, inplace=True)
    VAM_year = io_dict[year].value_added.D_imp_reg.T
    VAM_year.rename(columns={'Emissions': 'VAM'}, inplace=True)

    df_year = pd.concat([PE_year, CE_year, EEE_year, EEI_year, VAX_year, VAM_year], axis=1)
    df_year.insert(0, 'Year', year)

    df_year['NE'] = df_year['EEE'] - df_year['EEI']
    df_year['NV'] = df_year['VAX'] - df_year['VAM']
    df_year['EIE'] = df_year['EEE'] / df_year['VAX']
    df_year['EIM'] = df_year['EEI'] / df_year['VAM']
    df_year['NEI'] = df_year['EIE'] - df_year['EIM']

    # Append the DataFrame for the current year to the list
    dfs.append(df_year)

# Concatenate all DataFrames in the list into a single DataFrame
final_df = pd.concat(dfs, axis=0)

# Now final_df contains the combined data for years 2000 to 2014 with the calculated columns


In [ ]:
final_df

,Year,PE,CE,EEE,EEI,VAX,VAM,NE,NV,EIE,EIM,NEI
region,,,,,,,,,,,,
AUS,2000,345.316081,302.152261,108.707806,65.543987,2.172519e+05,8.891676e+04,43.163819,128335.086099,0.000500,0.000737,-0.000237
AUT,2000,58.441679,73.174187,29.080759,43.813267,8.027316e+04,7.782189e+04,-14.732508,2451.267161,0.000362,0.000563,-0.000201
BEL,2000,101.779061,120.766534,58.649285,77.636758,1.298563e+05,1.165020e+05,-18.987473,13354.375564,0.000452,0.000666,-0.000215
BGR,2000,46.368447,51.775700,5.797261,11.204514,2.009401e+03,9.376515e+03,-5.407253,-7367.113416,0.002885,0.001195,0.001690
BRA,2000,317.840130,354.709285,49.452942,86.322097,1.007506e+05,1.273335e+05,-36.869155,-26582.868454,0.000491,0.000678,-0.000187
...,...,...,...,...,...,...,...,...,...,...,...,...
SVN,2014,11.490613,12.646620,7.558566,8.714574,1.790762e+04,1.488508e+04,-1.156007,3022.537910,0.000422,0.000585,-0.000163
SWE,2014,41.934222,71.989270,24.137552,54.192599,1.475370e+05,1.172871e+05,-30.055048,30249.816143,0.000164,0.000462,-0.000298
TUR,2014,270.313594,298.797265,96.144523,124.628194,1.700001e+05,1.557449e+05,-28.483671,14255.209863,0.000566,0.000800,-0.000235


In [ ]:
# Extract rows with index value "IND" to get data on India
IND_rows = final_df.loc[final_df.index == "IND"]

In [ ]:
IND_rows

,Year,PE,CE,EEE,EEI,VAX,VAM,NE,NV,EIE,EIM,NEI
region,,,,,,,,,,,,
IND,2000,910.246273,836.244244,142.973373,68.971344,93206.083927,89161.190951,74.002029,4044.892975,0.001534,0.000774,0.000760
IND,2001,931.226727,873.158104,139.769412,81.700789,95753.898168,94971.830833,58.068622,782.067335,0.001460,0.000860,0.000599
IND,2002,956.244054,879.274570,164.525317,87.555833,110108.182732,103204.535756,76.969484,6903.646976,0.001494,0.000848,0.000646
IND,2003,978.213359,942.253544,146.452515,110.492701,105475.349856,125286.313939,35.959815,-19810.964083,0.001388,0.000882,0.000507
IND,2004,1050.198483,987.847448,201.965815,139.614780,146676.275604,149276.141107,62.351035,-2599.865503,0.001377,0.000935,0.000442
IND,2005,1111.334645,1052.007402,235.241069,175.913826,173743.063668,184456.872298,59.327243,-10713.808631,0.001354,0.000954,0.000400
IND,2006,1178.844715,1101.008923,260.173545,182.337753,200673.825611,212218.830628,77.835792,-11545.005018,0.001296,0.000859,0.000437
IND,2007,1280.517051,1227.670890,260.928206,208.082045,212572.717187,233596.672548,52.846162,-21023.955360,0.001227,0.000891,0.000337
IND,2008,1363.156990,1303.872711,263.419053,204.134774,207965.014285,235613.917331,59.284279,-27648.903047,0.001267,0.000866,0.000400


# **Sector-wise df (2000-2014)**




In [ ]:
# Initialize an empty list to store DataFrames for each year
dfs_SEC = []

# Iterate through years from 2000 to 2014
for year in range(2000, 2015):
    PE_SEC_year = io_dict[year].emissions.D_pba.T
    PE_SEC_year.rename(columns={'Emissions': 'PE'}, inplace=True)
    CE_SEC_year = io_dict[year].emissions.D_cba.T
    CE_SEC_year.rename(columns={'Emissions': 'CE'}, inplace=True)
    EEE_SEC_year = io_dict[year].emissions.D_exp.T
    EEE_SEC_year.rename(columns={'Emissions': 'EEE'}, inplace=True)
    EEI_SEC_year = io_dict[year].emissions.D_imp.T
    EEI_SEC_year.rename(columns={'Emissions': 'EEI'}, inplace=True)
    VAX_SEC_year = io_dict[year].value_added.D_exp.T
    VAX_SEC_year.rename(columns={'Emissions': 'VAX'}, inplace=True)
    VAM_SEC_year = io_dict[year].value_added.D_imp.T
    VAM_SEC_year.rename(columns={'Emissions': 'VAM'}, inplace=True)

    df_SEC_year = pd.concat([PE_SEC_year, CE_SEC_year, EEE_SEC_year, EEI_SEC_year, VAX_SEC_year, VAM_SEC_year], axis=1)
    df_SEC_year.insert(0, 'Year', year)

    df_SEC_year['NE'] = df_SEC_year['EEE'] - df_SEC_year['EEI']
    df_SEC_year['NV'] = df_SEC_year['VAX'] - df_SEC_year['VAM']
    df_SEC_year['EIE'] = df_SEC_year['EEE'] / df_SEC_year['VAX']
    df_SEC_year['EIM'] = df_SEC_year['EEI'] / df_SEC_year['VAM']
    df_SEC_year['NEI'] = df_SEC_year['EIE'] - df_SEC_year['EIM']

    # Append the DataFrame for the current year to the list
    dfs_SEC.append(df_SEC_year)

# Concatenate all DataFrames in the list into a single DataFrame
final_df_SEC = pd.concat(dfs_SEC, axis=0)

# Now final_df_SEC contains the combined data for years 2000 to 2014 with the calculated columns


In [ ]:
final_df_SEC

Year           PE           CE         EEE         EEI  \
region sector                                                           
AUS    Sec1    2000     5.309037     3.018561    2.414508    0.699658   
       Sec2    2000    10.938212     3.543781    8.656289    0.626076   
       Sec3    2000    73.618097    57.447608   39.414302   29.597553   
       Sec8    2000    27.718158   102.322108    4.009416   22.089288   
       Sec4    2000   179.806378    73.072121   41.213717    0.805825   
...             ...          ...          ...         ...         ...   
USA    Sec8    2014   708.842119  1996.755043   35.677459  374.743054   
       Sec4    2014  2094.482887  1119.934677   94.097492   13.762101   
       Sec5    2014    62.942855   242.407991    0.491808   74.658296   
       Sec6    2014   505.834145   263.484148  109.354314   39.498502   
       Sec7    2014    61.084679   160.019628    1.227434   25.819692   

                         VAX            VAM          NE             NV  \
region sector                                                            
AUS    Sec1     11243.518059     980.935589    1.714850   10262.582471   
       Sec2     70922.157173     773.160926    8.030213   70148.996247   
       Sec3     36176.105769   44243.008271    9.816750   -8066.902503   
       Sec8     76448.414704   28385.646015  -18.079872   48062.768689   
       Sec4      3738.196066     942.258599   40.407893    2795.937467   
...                      ...            ...         ...            ...   
USA    Sec8    717471.197829  590494.221764 -339.065595  126976.976065   
       Sec4     11308.511208   15150.295072   80.335391   -3841.783863   
       Sec5      4607.691633   82459.775327  -74.166488  -77852.083695   
       Sec6     57645.413702   38747.050424   69.855812   18898.363278   
       Sec7      9218.172607   38281.718464  -24.592259  -29063.545857   

                    EIE       EIM       NEI  
region sector                                
AUS    Sec1    0.000215  0.000713 -0.000499  
       Sec2    0.000122  0.000810 -0.000688  
       Sec3    0.001090  0.000669  0.000421  
       Sec8    0.000052  0.000778 -0.000726  
       Sec4    0.011025  0.000855  0.010170  
...                 ...       ...       ...  
USA    Sec8    0.000050  0.000635 -0.000585  
       Sec4    0.008321  0.000908  0.007413  
       Sec5    0.000107  0.000905 -0.000799  
       Sec6    0.001897  0.001019  0.000878  
       Sec7    0.000133  0.000674 -0.000541  

[5160 rows x 12 columns]

In [ ]:
# Extract rows with index value "IND" to get data on India
IND_SEC_rows = final_df_SEC.query("region == 'IND'")

In [ ]:
IND_SEC_rows

Year          PE          CE         EEE        EEI  \
region sector                                                        
IND    Sec1    2000   52.707855   79.001214    3.860092   2.641385   
       Sec2    2000   47.716225  -20.887426   14.926082  -0.324068   
       Sec3    2000  251.972922  355.272136   37.233139  40.291613   
       Sec8    2000   22.773726   81.540332    3.875523   7.917498   
       Sec4    2000  446.006841   88.653865   65.247754   0.547021   
...             ...         ...         ...         ...        ...   
       Sec8    2014   53.030588  159.192490    7.950952  17.878726   
       Sec4    2014  924.740898  184.112896  144.748104   2.167069   
       Sec5    2014   14.483864  427.714315    0.275988  45.825226   
       Sec6    2014   84.134765  145.122677   17.603288  19.273624   
       Sec7    2014   40.099027   54.894849    2.216599   2.809082   

                         VAX           VAM          NE             NV  \
region sector                                                           
IND    Sec1     15482.819108   3453.445316    1.218706   12029.373793   
       Sec2      8603.962850   -437.665289   15.250150    9041.628139   
       Sec3     19919.030768  51366.739246   -3.058474  -31447.708478   
       Sec8     38485.109359  13914.325441   -4.041974   24570.783918   
       Sec4      1885.117785    762.652815   64.700733    1122.464970   
...                      ...           ...         ...            ...   
       Sec8    139593.467880  23039.187467   -9.927774  116554.280413   
       Sec4      5050.610595   2597.974156  142.581036    2452.636440   
       Sec5      2709.717924  46714.313690  -45.549238  -44004.595766   
       Sec6     15924.526326  23291.213165   -1.670336   -7366.686839   
       Sec7      1606.370225   4070.252954   -0.592484   -2463.882729   

                    EIE       EIM       NEI  
region sector                                
IND    Sec1    0.000249  0.000765 -0.000516  
       Sec2    0.001735  0.000740  0.000994  
       Sec3    0.001869  0.000784  0.001085  
       Sec8    0.000101  0.000569 -0.000468  
       Sec4    0.034612  0.000717  0.033895  
...                 ...       ...       ...  
       Sec8    0.000057  0.000776 -0.000719  
       Sec4    0.028660  0.000834  0.027825  
       Sec5    0.000102  0.000981 -0.000879  
       Sec6    0.001105  0.000828  0.000278  
       Sec7    0.001380  0.000690  0.000690  

[120 rows x 12 columns]

# **Save files to excel**

In [ ]:
IND_rows.to_excel('/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Graphs/IND_rows.xlsx')

In [ ]:
IND_SEC_rows.to_excel('/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Graphs/IND_SEC_rows.xlsx')